In [4]:
pip install easyocr pdf2image Pillow

  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 2.9/2.9 MB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 38.9/38.9 MB 1.8 MB/s eta 0:00:00
Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
   ---------------------------------------- 12.8/12.8 MB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 7.0/7.0 MB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 41.3/41.3 MB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 1.7/1.7 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.0
    Uninstalling scipy-1.10.

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


In [9]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install sentence-transformers faiss-cpu numpy

Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load your chunked data
with open("chunked.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# Load multilingual embedding model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings
print("Generating embeddings...")
embeddings = model.encode(chunks, show_progress_bar=True)

# Convert to FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 = Euclidean distance
index.add(np.array(embeddings))

# Save FAISS index
faiss.write_index(index, "faiss_index.index")

# Save metadata (to map index to chunks later)
with open("index_metadata.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print("Vector store created and saved successfully!")

Generating embeddings...


Batches: 100%|██████████| 7/7 [00:21<00:00,  3.09s/it]

✅ Vector store created and saved successfully!


In [11]:
!pip install transformers
!pip install torch

In [ ]:
!python -m venv mt5env
!mt5env\Scripts\activate  # For Windows
!pip install --upgrade pip

!pip install torch
!pip install transformers
!pip install sentencepiece
!pip install faiss-cpu
!pip install sentence-transformers

Error: [Errno 13] Permission denied: 'd:\\OneDrive - Universiti Teknologi Malaysia (UTM)\\Study Materials\\MY PROJECTS\\rag-system\\mt5env\\Scripts\\python.exe'
'mt5env\Scripts\activate' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import json
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer

# Load the chunked data
with open("index_metadata.json", "r", encoding="utf-8") as f:
    chunk_texts = json.load(f)

# Load FAISS vector index
index = faiss.read_index("faiss_index.index")

# Load multilingual embedding model
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Load mT5 model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

def retrieve_and_answer_mt5(query, k=3):
    # Embed query
    query_vec = embedding_model.encode([query])
    
    # Search top k similar chunks
    D, I = index.search(np.array(query_vec), k)
    retrieved_chunks = [chunk_texts[i] for i in I[0]]
    context = "\n".join(retrieved_chunks)

    # Build prompt
    prompt = f"""প্রসঙ্গ:
{context}

প্রশ্ন: {query}
উত্তর:"""

    # Tokenize prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=100,
            do_sample=False,
            early_stopping=True
        )

    # Decode and return
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

print("\nBengali RAG System (mT5)")
print("টাইপ করুন আপনার প্রশ্ন (exit লিখলে বন্ধ হবে)\n")

while True:
    query = input("প্রশ্ন: ")
    if query.strip().lower() in ["exit", "quit", "q"]:
        print("বিদায়!")
        break

    answer = retrieve_and_answer_mt5(query)
    print("উত্তর:", answer)

C:\Users\abdul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(



Bengali RAG System (mT5)
❓ টাইপ করুন আপনার প্রশ্ন (exit লিখলে বন্ধ হবে)



The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


উত্তর: <extra_id_0> ি ্য্য্য্য্য্য


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


উত্তর: <extra_id_0> ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য ্য
👋 বিদায়!
